<h1>Importing</h1>

In [4]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from tqdm import tqdm
import tensorflow as tf

2024-08-02 16:56:21.330749: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-02 16:56:21.341940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-02 16:56:21.359872: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-02 16:56:21.365184: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-02 16:56:21.378042: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instru

In [5]:
!nvidia-smi

Fri Aug  2 16:56:23 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           On  |   00000000:82:00.0 Off |                    0 |
| N/A   32C    P0             25W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [10]:
train_dataset_path = "train.csv"
sample_dataset_path = "sample_submission.csv"
test_dataset_path = "test-2.csv"
df = pd.read_csv(train_dataset_path)
df.head()

,img_path,printer_id,print_id,has_under_extrusion
0,101/1678589738/1678589914.060332.jpg,101,1678589738,1
1,101/1678589738/1678589914.462857.jpg,101,1678589738,1
2,101/1678589738/1678589914.875075.jpg,101,1678589738,1
3,101/1678589738/1678589915.284495.jpg,101,1678589738,1
4,101/1678589738/1678589915.700486.jpg,101,1678589738,1


In [8]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        image_path = self.df.loc[index]['img_path']
        label = self.df.iloc[index][-1].astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, image_path))
        if self.transform is not None:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        return image, label

In [11]:
transform = transforms.Compose([
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_val_dataset = CustomDataset(csv_path = train_dataset_path, images_folder = "./images", transform = transform)
test_dataset = CustomDataset(csv_path = test_dataset_path, images_folder = "./images", transform = transform)

train_dataset,validation_dataset=torch.utils.data.random_split(train_val_dataset, [0.75,0.25],)


In [12]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
validation_loader = DataLoader(validation_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = True)


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [20]:
model = models.efficientnet_b0(pretrained=True)

In [16]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
size = len(train_dataset)
epochs = 5

AttributeError: 'collections.OrderedDict' object has no attribute 'parameters'

In [ ]:
for params in model.parameters():
    params.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

In [ ]:
model.to(device)

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward Pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        
        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = (outputs > 0).float()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)
    
    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total

print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
print("Finished Training")

In [ ]:
# # export model weights
# torch.save(model.state_dict(), 'model.pth')

In [ ]:
model.eval()  # Set the model to evaluation mode

test_correct = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # binary classification
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
    
    test_accuracy = 100 * test_correct / test_total

print(f'Test Accuracy: {test_accuracy}')